In [10]:
import numpy as np
import re
import pandas as pd
from konlpy.tag import Mecab
from soynlp.normalizer import *

# 한국어 감정 정보가 포함된 단발성 대화 데이터셋

In [11]:
df1 = pd.read_excel("/home/wonhong/workspace/Emotion_Detection/DL/dataset/한국어_단발성_대화_데이터셋.xlsx")

In [13]:
# 놀람 > 중립, 혐오 당황 등의 감정과 혼동이 되므로, 놀람 삭제
df1 = df1.loc[df1['Emotion']!='놀람', ['Sentence', 'Emotion']]

In [14]:
# 공포 > 공포보다는 '불안'에 가까움
df1['Emotion'] = np.where(df1['Emotion'].str.match('공포'), '불안', df1['Emotion'])

In [30]:
df1.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,불안
1,그냥 내 느낌일뿐겠지?,불안
2,아직너무초기라서 그런거죠?,불안
3,유치원버스 사고 낫다던데,불안
4,근데 원래이런거맞나요,불안


# 감성대화 말뭉치 - xlxs

In [16]:
df_train = pd.read_excel('/home/wonhong/workspace/Emotion_Detection/DL/dataset/감성대화말뭉치(최종데이터)_Training.xlsx')
df_val = pd.read_excel('/home/wonhong/workspace/Emotion_Detection/DL/dataset/감성대화말뭉치(최종데이터)_Validation.xlsx')

In [18]:
df_train = df_train.fillna("")
df_val = df_val.fillna("")

In [21]:
df_train['사람문장'] = df_train['사람문장1'].astype(str) + df_train['사람문장2'].astype(str) + df_train['사람문장3'].astype(str)
df_val['사람문장'] = df_val['사람문장1'].astype(str) + df_val['사람문장2'].astype(str) + df_val['사람문장3'].astype(str)

In [22]:
df2 = pd.concat([df_train, df_val], axis=0)

In [24]:
df2 = df2[['사람문장', '감정_대분류']]
df2.head()

,사람문장,감정_대분류
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,분노


In [25]:
df2 = df2.rename({'감정_대분류' : '감정'}, axis=1)

In [27]:
df2['감정'] = df2['감정'].apply(lambda x: x.strip())

In [28]:
df2['감정'].value_counts()

불안    10433
분노    10417
상처    10150
슬픔    10128
당황     9804
기쁨     7339
Name: 감정, dtype: int64

In [29]:
# 상처 > 슬픔
# 기쁨 > 행복

df2['감정'] = np.where(df2['감정'].str.match('상처'), '슬픔', df2['감정'])
df2['감정'] = np.where(df2['감정'].str.match('기쁨'), '행복', df2['감정'])

In [32]:
df2 = df2.rename({'사람문장' : 'Sentence', '감정' : 'Emotion'}, axis=1)

# 한국어 연속적 대화 데이터셋

In [39]:
df3 = pd.read_excel("/home/wonhong/workspace/Emotion_Detection/DL/dataset/한국어_연속적_대화_데이터셋.xlsx")

In [43]:
new_col = df3.iloc[0]

df3 = df3.rename(columns=new_col).iloc[:, 1:3]

In [46]:
df3 = df3.loc[df3['감정'].isin(['중립, 놀람, 분노', '슬픔', '행복', '혐오', '공포'])]

In [47]:
# 놀람 > 당황
# 공포 > 불안

df3['감정'] = np.where(df3['감정'].str.match('놀람'), '당황', df3['감정'])
df3['감정'] = np.where(df3['감정'].str.match('공포'), '불안', df3['감정'])

In [53]:
df3 = df3.rename({'발화' : 'Sentence', '감정' : 'Emotion'}, axis=1)

---

# merge all

In [55]:
df = pd.concat([df1, df2, df3]).reset_index(drop=True)

In [56]:
df.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,불안
1,그냥 내 느낌일뿐겠지?,불안
2,아직너무초기라서 그런거죠?,불안
3,유치원버스 사고 낫다던데,불안
4,근데 원래이런거맞나요,불안


In [58]:
df['Emotion'].value_counts()

슬픔    27517
분노    16082
불안    15999
행복    14406
당황     9804
혐오     5649
중립     4830
Name: Emotion, dtype: int64

In [60]:
df.to_csv('dataset/merged_data.csv', index=False)